# Rock Paper Scissor Image Classification

oleh : Ricky Ariansyah 

email : rickyarians@outlook.com

Dataset : Rock Paper Scissors
Total Images : 2188
Rock : 726 | Paper : 712 | Scissors : 750

Accuracy

 Acc : 93%

## Tahapan

- memastikan library yang akan dipakai

- Mengunduh dataset dan melakukan extract file dengan metode unzip.

- Menampung direktori setiap kelas pada direktori train dan validasi ke dalam variabel.

- Pre-processing data dengan image augmentation.

- Mempersiapkan data latih yang akan dipelajari oleh model.

- Membangun arsitektur model dengan Convolutional Neural Network (CNN).
- Compile dan latih model dengan model.compile dan model.fit hingga mendapatkan akurasi yang diinginkan.

- Menguji model yang telah dibuat dengan menggunakan gambar yang belum dikenali oleh model.

## Kriteria Submission

- [x] Dataset yang dipakai haruslah dataset berikut : rockpaperscissors, atau gunakan link ini pada wget command: https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip.
- Dataset harus dibagi menjadi train set dan validation set.
- Ukuran validation set harus 40% dari total dataset (data training memiliki 1314 sampel, dan data validasi sebanyak 874 sampel).
- Harus mengimplementasikan augmentasi gambar.
- Menggunakan image data generator.
- Model harus menggunakan model sequential.
- Pelatihan model tidak melebihi waktu 30 menit.
- Program dikerjakan pada Google Colaboratory.
- Akurasi dari model minimal 85%.

### Import Library yang dibutuhkan

In [ ]:
# library untuk prroses file berbentuk zip
import zipfile
# library menggunakan fungsi operating system
import os
import time

# library machine learning tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dense, Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# libarary tambahan untuk visualisasi, proses image, proses data dan google colab
import keras.utils as image
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

### Download Dataset



In [ ]:
# Download dataset menggunakan wget
!wget https://github.com/dicodingacademy/assets/releases/download/release/rockpaperscissors.zip

### Unzip / Ekstrak Dataset

ekstrak dataset yang telah didownload menggunakan library zipfile

In [ ]:
# Ekstrak file tipe zip
ekstrak_zip = 'rockpaperscissors.zip'
out_zip = zipfile.ZipFile(ekstrak_zip, 'r')
# ekstrak ke dalam folder dataset
out_zip.extractall('/dataset')
out_zip.close()

### Atur Lokasi Direktori Dataset RockPaperScissors

atur lokasi direktori dan masukkan ke dalam variabel untuk mempermudah pemanggilan dataset

In [ ]:
dir_dataset = "/dataset/rockpaperscissors/rps-cv-images"
dir_paper = os.path.join("/dataset/rockpaperscissors/rps-cv-images/paper")
dir_rock = os.path.join("/dataset/rockpaperscissors/rps-cv-images/rock")
dir_scissors = os.path.join("/dataset/rockpaperscissors/rps-cv-images/scissors")

### cek hasil ekstrak

In [ ]:
from posix import listdir
print(f"{listdir('/dataset/rockpaperscissors')}")

### cek total data

In [ ]:
# cek total data


print(f"Total Data Image paper : {len(os.listdir(dir_paper))}")
print(f"Total Data Image rock : {len(os.listdir(dir_rock))}")
print(f"Total Data Image scissors : {len(os.listdir(dir_scissors))}")

print(f"Total Data Image keseluruhan {len(os.listdir(dir_paper)) + len(os.listdir(dir_rock)) + len(os.listdir(dir_scissors))}")

### Image data generator


kita akan menerapkan ImageDataGenerator untuk data latih dan data validasi. 

In [ ]:
# deklarasi variabel size validasi sesuai ketentuan sebanyak 40%
val_size = 0.4

Train_datagen = ImageDataGenerator(
    rotation_range = 30,
    brightness_range = [0.2,1.0],
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest",
    rescale = 1./255,
    validation_split = val_size
)

Validation_datagen = ImageDataGenerator(
    rotation_range = 30,
    brightness_range = [0.2,1.0],
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = "nearest",
    rescale = 1./255,
    validation_split = val_size
)



### Split Dataset

Ukuran validation set harus 40% dari total dataset (data training memiliki 1314 sampel, dan data validasi sebanyak 874 sampel).

In [ ]:
# Hasil split dataset dengan 
# Train : 1314 | Validation :  874

# Mengatur lebar dan tinggi gambar
img_width = 150
img_height = 150

# Train dan Validation generator  dengan mode categorical karena klasifikasi ini lebih dari 2
Train_generator = Train_datagen.flow_from_directory(
    dir_dataset,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 16,
    shuffle = True,
    subset = "training"
)

Validation_generator = Validation_datagen.flow_from_directory(
    dir_dataset,
    target_size = (img_width,img_height),
    color_mode = "rgb",
    class_mode = "categorical",
    batch_size = 16,
    shuffle = False,
    subset = "validation"
)

In [ ]:
# Pembuatan Model dengan tipe Sequential

Model = Sequential(
    [
     Conv2D(32, (3,3), strides = (1,1), activation = 'relu' , input_shape = (img_width,img_height,3)),
     MaxPooling2D(pool_size = (2,2), padding = 'valid'),
     Conv2D(64, (3,3), strides = (1,1), activation = 'relu' ),
     MaxPooling2D(pool_size = (2,2), padding = 'valid'),
     Conv2D(128, (3,3), strides = (1,1), activation = 'relu' ),
     MaxPooling2D(pool_size = (2,2), padding = 'valid'),
     Flatten(),
     Dropout(0.2),
     Dense(128, activation = 'relu'),
     Dense(3, activation='softmax')
    ]
)

In [ ]:
# Penggunaan Optimizer 'Adam'
Adam(name='Adam')
Model.compile(optimizer = 'Adam',loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [ ]:

# Penggunaan Callbacks dengan fungsi EarlyStopping dengan memonitoring Val_Loss
callback = EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 3,
    verbose = 1,
    mode = 'auto'
)
start = time.time()
Model.fit(
    Train_generator,
    epochs = 10,
    validation_data = Validation_generator,
    callbacks=[callback]
)
stop = time.time()
print(f"Training time: {round((stop - start)/60)}minute")
     

In [ ]:
# fungsi kondisi

def predict_image(image):
    classes = Model.predict(images, batch_size=16)
    print("\n")
    print('Hasil Prediksi : ',classes[0],'\n')

    if classes[0][0] == 1:
      print('Hasil Gambar : Paper')
    elif classes[0][1] == 1:
      print('Hasil Gambar : Rock')
    else:
      print('Hasil Gambar : Scissor')


  

In [ ]:
# Upload File ke google colabs
uploaded = files.upload()

for file_upload in uploaded.keys():

  path = file_upload
  img = image.load_img(path, target_size=(150,150))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  # Membuat numpy vstack array untuk hasil prediksi 
  images = np.vstack([x])
 
  predict_image(images)